In [1]:

from pyspark import SparkContext
from pyspark.sql import HiveContext
from pyspark import SparkConf

# Create the SparkContext object and the SQLContext or HiveContext object
#sc = SparkContext()
#sqlContext = HiveContext(sc)

import matplotlib
matplotlib.use('PDF')

import pandas as pd
import pyspark.sql.functions as F
from pyspark.sql.functions import unix_timestamp



In [2]:
fullpromodat_1=sqlContext.sql( """ select * from raleys_analyst.jn_fullpromodat_1_2018Q3 """)

In [3]:
fullpromodat_1.show()

+--------------+----------+--------------+-------+--------+------------------+-------------------+-------------------+-------------------+---------------+--------------+------------+----------------+---------------+----------+
|     prod_code|store_code|offer_tpr_code|prod_id|store_id|prod_internal_code|     tpr_start_dttm|       tpr_end_dttm|offer_tpr_type_code|offer_price_amt|offer_prod_qty|markdown_pct|normal_price_amt|normal_prod_qty|unit_price|
+--------------+----------+--------------+-------+--------+------------------+-------------------+-------------------+-------------------+---------------+--------------+------------+----------------+---------------+----------+
|00000000003294|       110|         M6025|  34267|    1300|          10700110|2017-11-08 05:00:00|2018-01-02 05:00:00|                  3|          2.490|         1.000|       0.000|           3.290|          1.000|     3.290|
|00000000003628|       233|         M6025| 117918|    1453|          10700593|2017-11-08 05:

In [4]:
promodat_1 = fullpromodat_1.withColumn('base_price', (F.col('NORMAL_PRICE_AMT')/F.col('NORMAL_PROD_QTY')))

In [5]:
promodat_1.show()

+--------------+----------+--------------+-------+--------+------------------+-------------------+-------------------+-------------------+---------------+--------------+------------+----------------+---------------+----------+--------------------+
|     prod_code|store_code|offer_tpr_code|prod_id|store_id|prod_internal_code|     tpr_start_dttm|       tpr_end_dttm|offer_tpr_type_code|offer_price_amt|offer_prod_qty|markdown_pct|normal_price_amt|normal_prod_qty|unit_price|          base_price|
+--------------+----------+--------------+-------+--------+------------------+-------------------+-------------------+-------------------+---------------+--------------+------------+----------------+---------------+----------+--------------------+
|00000000003294|       110|         M6025|  34267|    1300|          10700110|2017-11-08 05:00:00|2018-01-02 05:00:00|                  3|          2.490|         1.000|       0.000|           3.290|          1.000|     3.290|3.290000000000000...|
|0000000

In [6]:
promodat_1 = promodat_1.where(F.col('normal_price_amt')>=F.col('offer_price_amt'))

In [7]:
promodat_1.show()

+--------------+----------+--------------+-------+--------+------------------+-------------------+-------------------+-------------------+---------------+--------------+------------+----------------+---------------+----------+--------------------+
|     prod_code|store_code|offer_tpr_code|prod_id|store_id|prod_internal_code|     tpr_start_dttm|       tpr_end_dttm|offer_tpr_type_code|offer_price_amt|offer_prod_qty|markdown_pct|normal_price_amt|normal_prod_qty|unit_price|          base_price|
+--------------+----------+--------------+-------+--------+------------------+-------------------+-------------------+-------------------+---------------+--------------+------------+----------------+---------------+----------+--------------------+
|00000000003294|       110|         M6025|  34267|    1300|          10700110|2017-11-08 05:00:00|2018-01-02 05:00:00|                  3|          2.490|         1.000|       0.000|           3.290|          1.000|     3.290|3.290000000000000...|
|0000000

In [8]:
from pyspark.sql.functions import *

In [9]:
promodat_2 = promodat_1.withColumn('sale_price', when(col('offer_tpr_type_code') == 8, round(col('base_price')/2,2))  
                                   .when(((col('offer_price_amt') > 0) & (col('offer_prod_qty') > 0))
                                   , round(col('offer_price_amt')/col('offer_prod_qty'),2))
                                   .when(((col('markdown_pct') > 0) & (col('base_price') > 0)) #then use markdown percentage and base price
                                   , (col('base_price') - (round((col('markdown_pct')/100)*col('base_price'),2)))) 
                                   .when(((col('markdown_pct') > 0) & (col('base_price').isNull()))  #no base price then use unit price
                                  , (col('unit_price') - (round((col('markdown_pct')/100)*col('unit_price'),2)))))


In [10]:
promodat_2.show()

+--------------+----------+--------------+-------+--------+------------------+-------------------+-------------------+-------------------+---------------+--------------+------------+----------------+---------------+----------+--------------------+--------------------+
|     prod_code|store_code|offer_tpr_code|prod_id|store_id|prod_internal_code|     tpr_start_dttm|       tpr_end_dttm|offer_tpr_type_code|offer_price_amt|offer_prod_qty|markdown_pct|normal_price_amt|normal_prod_qty|unit_price|          base_price|          sale_price|
+--------------+----------+--------------+-------+--------+------------------+-------------------+-------------------+-------------------+---------------+--------------+------------+----------------+---------------+----------+--------------------+--------------------+
|00000000003294|       110|         M6025|  34267|    1300|          10700110|2017-11-08 05:00:00|2018-01-02 05:00:00|                  3|          2.490|         1.000|       0.000|           

In [11]:
promodat_3 = promodat_2.withColumn('discount_amt', when((col('offer_tpr_type_code') == 8), col('sale_price'))   
                                   .when(((col('offer_price_amt') > 0) & (col('offer_prod_qty') > 0))
                                   , (col('base_price') - col('sale_price')))                                    
                                   .when(((col('markdown_pct') > 0) & (col('base_price') > 0))
                                   , (round((col('markdown_pct')/100) * (col('base_price')),2)))
                                   .when(((col('markdown_pct') > 0) & (col('base_price').isNull()))
                                   , (round((col('markdown_pct')/100) * (col('unit_price')),2))))

In [12]:
promodat_3.show()

+--------------+----------+--------------+-------+--------+------------------+-------------------+-------------------+-------------------+---------------+--------------+------------+----------------+---------------+----------+--------------------+--------------------+--------------------+
|     prod_code|store_code|offer_tpr_code|prod_id|store_id|prod_internal_code|     tpr_start_dttm|       tpr_end_dttm|offer_tpr_type_code|offer_price_amt|offer_prod_qty|markdown_pct|normal_price_amt|normal_prod_qty|unit_price|          base_price|          sale_price|        discount_amt|
+--------------+----------+--------------+-------+--------+------------------+-------------------+-------------------+-------------------+---------------+--------------+------------+----------------+---------------+----------+--------------------+--------------------+--------------------+
|00000000003294|       110|         M6025|  34267|    1300|          10700110|2017-11-08 05:00:00|2018-01-02 05:00:00|            

In [13]:
promodat_4 = promodat_3.withColumn('discount_pct', when(col('offer_tpr_type_code') == 8, 0.50) 
                                   .when(((col('offer_price_amt') > 0) & (col('offer_prod_qty') > 0))
                                   , round((col('base_price') - col('sale_price'))/ col('base_price'),2)) 
                                   .when(((col('markdown_pct') > 0) & (col('base_price') > 0))
                                   , round((col('base_price') - col('sale_price'))/ col('base_price'),2))
                                   .when(((col('markdown_pct') > 0) & (col('base_price').isNull()))
                                   , round((col('unit_price') - col('sale_price'))/ col('unit_price'),2)))

In [14]:
promodat_4.show()

+--------------+----------+--------------+-------+--------+------------------+-------------------+-------------------+-------------------+---------------+--------------+------------+----------------+---------------+----------+--------------------+--------------------+--------------------+------------+
|     prod_code|store_code|offer_tpr_code|prod_id|store_id|prod_internal_code|     tpr_start_dttm|       tpr_end_dttm|offer_tpr_type_code|offer_price_amt|offer_prod_qty|markdown_pct|normal_price_amt|normal_prod_qty|unit_price|          base_price|          sale_price|        discount_amt|discount_pct|
+--------------+----------+--------------+-------+--------+------------------+-------------------+-------------------+-------------------+---------------+--------------+------------+----------------+---------------+----------+--------------------+--------------------+--------------------+------------+
|00000000003294|       110|         M6025|  34267|    1300|          10700110|2017-11-08 05

In [15]:
promodat_5 = promodat_4.withColumn('mechanic', when(col('offer_tpr_type_code') == 8, lit('BOGO')) 
                                   .when((((col('offer_price_amt') > 0) & (col('offer_prod_qty') > 0)))
                                    & (col('offer_prod_qty') == 1) & (col('discount_pct') == 0),lit("Base_Price"))
                                   .when((((col('offer_price_amt') > 0) & (col('offer_prod_qty') > 0))) 
                                    & (col('offer_prod_qty') == 1) & ((col('discount_pct') > 0) & (col('discount_pct') < 0.15)),lit("Shallow")) 
                                   .when((((col('offer_price_amt') > 0) & (col('offer_prod_qty') > 0)))
                                   & (col('offer_prod_qty') == 1) & ((col('discount_pct') >= 0.15) & (col('discount_pct') < 0.30)) ,lit("Medium"))
                                   .when((((col('offer_price_amt') > 0) & (col('offer_prod_qty') > 0)))
                                   & (col('offer_prod_qty') == 1) & ((col('discount_pct') >= 0.30)) ,lit("Deep"))                                   
                                   .when((((col('offer_price_amt') > 0) & (col('offer_prod_qty') > 0))) & (col('offer_prod_qty') > 1)
                                   ,concat(round(col('offer_prod_qty'),0).cast('string') , lit(' for '), round(col('offer_price_amt'),0).cast('string') ))
                                   .when(col('markdown_pct') > 0, concat(round(col('markdown_pct'),0).cast('string'), lit('percent_off'))))

In [16]:
promodat_5.show()

Py4JJavaError: An error occurred while calling o291.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 6.0 failed 4 times, most recent failure: Lost task 0.3 in stage 6.0 (TID 9, usloubd5r01s10.dunnhumby.co.uk, executor 2): org.codehaus.janino.JaninoRuntimeException: failed to compile: org.codehaus.janino.JaninoRuntimeException: Code of method "apply_7$(Lorg/apache/spark/sql/catalyst/expressions/GeneratedClass$SpecificUnsafeProjection;Lorg/apache/spark/sql/catalyst/InternalRow;)V" of class "org.apache.spark.sql.catalyst.expressions.GeneratedClass$SpecificUnsafeProjection" grows beyond 64 KB
/* 001 */ public java.lang.Object generate(Object[] references) {
/* 002 */   return new SpecificUnsafeProjection(references);
/* 003 */ }
/* 004 */
/* 005 */ class SpecificUnsafeProjection extends org.apache.spark.sql.catalyst.expressions.UnsafeProjection {
/* 006 */
/* 007 */   private Object[] references;
/* 008 */   private UTF8String.IntWrapper wrap

In [ ]:
promodat_3 = promodat_3.withColumn('mechanic_desc', when(col('mechanic')=="2 for 1",lit("BOGO"))
                                   .when(col('offer_prod_qty') ==1, lit("Discount"))
                                   .when(col('offer_prod_qty') > 1, lit("Multiple"))
                                   .when (col('markdown_pct') > 0, lit('percent_off')))


In [ ]:
promodat_3 = promodat_3.withColumn('mechanic_price', when(col('mechanic')=="2 for 1",lit("BOGO"))
                                   .when((((col('offer_price_amt') > 0) & (col('offer_prod_qty') > 0))) 
                                    & (col('offer_prod_qty') == 1) & ((col('discount_pct') > 0) & (col('discount_pct') < 0.15)),lit("Shallow")) 
                                   .when((((col('offer_price_amt') > 0) & (col('offer_prod_qty') > 0)))
                                   & (col('offer_prod_qty') == 1) & ((col('discount_pct') >= 0.15) & (col('discount_pct') < 0.30)) ,lit("Medium"))
                                   .when((((col('offer_price_amt') > 0) & (col('offer_prod_qty') > 0)))
                                   & (col('offer_prod_qty') == 1) & ((col('discount_pct') >= 0.30)) ,lit("Deep"))
                                   .when((((col('offer_price_amt') > 0) & (col('offer_prod_qty') > 0)))
                                    & (col('offer_prod_qty') == 1) & (col('discount_pct') == 0),lit("Base_Price"))
                                   .when(col('offer_prod_qty') > 1, lit("Multiple"))
                                   .when (col('markdown_pct') > 0, lit('percent_off')))


In [ ]:
promodat_3.show()

In [ ]:
promo_qa=promodat_3.groupBy("prod_code","tpr_start_dttm", "tpr_end_dttm").agg(F.countDistinct("prod_code").alias("flag"))

In [ ]:
promodat_3_nodups=promodat_3.dropDuplicates()

In [ ]:
promodat_3_nodups.write.saveAsTable('raleys_analyst.jn_promodat_3_nodups_2018Q3', mode='overwrite')
